In [1]:
from pathlib import Path
from IPython.display import HTML, display
css = Path("../../../css/rtl.css").read_text(encoding="utf-8")
display(HTML(f"<style>{css}</style>"))

# فصل ۳ — تحلیل اکتشافی داده (EDA)
## درس ۹: کالبدشکافی نشت اطلاعات در EDA (ویژگی‌های مشکوک، متغیرهای پس از وقوع نتیجه)

**در این درس چه چیزهایی یاد می‌گیرید**

- تعریف دقیق «نشت اطلاعات» و تمایز آن از «سیگنال معتبر».
- استفاده از EDA برای آشکارسازی نشت *قبل از* آن‌که تصمیم‌های مدل‌سازی پرهزینه شوند.
- شناسایی ویژگی‌های مشکوک با ابزارهای سریع و تکرارپذیر (غربال تک‌ویژگی، امتیاز نشت، حساسیت به نوع برش داده).
- تشخیص متغیرهای «پس از وقوع نتیجه» و تحمیل قرارداد «زمانِ در دسترس بودن» برای ویژگی‌ها.
- اعتبارسنجی ظن نشت با آزمایش‌های کنترل‌شده: تغییر نوع Split، حذف ستون‌ها، و بررسی افت عملکرد.
- ساخت یک چک‌لیست قابل استفادهٔ مجدد برای کالبدشکافی نشت در هر دیتاست جدید.

**دیتاست‌های مورد استفاده در این درس (از داخل ریپو)**

برای تمرین نشت در سناریوهای مختلف، چند دیتاست را به‌صورت هم‌زمان استفاده می‌کنیم:

- طبقه‌بندی: `../../../Datasets/Classification/diabetes.csv`
- رگرسیون: `../../../Datasets/Regression/house-prices.csv`
- دادهٔ جدولی واقعی با ستون‌های زمانی/فرآیندی: `../../../Datasets/Regression/listings.csv`
- دادهٔ فرآیندی (شکایات): `../../../Datasets/Clustering/ConsumerComplaints.csv`

## ۱) تعریف دقیق نشت اطلاعات؛ تعریفی که واقعاً به کار می‌آید

در عملِ یادگیری ماشین، **نشت اطلاعات** یعنی دادهٔ آموزشی شما (ویژگی‌ها، پیش‌پردازش، یا پروتکل ارزیابی) حاوی اطلاعاتی است که در زمان واقعیِ پیش‌بینی **در دسترس نیست**.

یک روش عملی برای دقیق‌کردن این مفهوم، تعریف **مرزِ در دسترس بودن** است:

- $t_0$ را «زمانِ پیش‌بینی» در نظر بگیرید (زمانی که می‌خواهید یک نمونهٔ جدید را امتیازدهی کنید).
- ویژگی $x_j$ تنها زمانی معتبر است که بتوان آن را با اطلاعات موجود در زمان $t \le t_0$ محاسبه کرد.
- نشت زمانی رخ می‌دهد که هر بخشی از $x$ (یا پایپلاین تولیدکنندهٔ $x$) از اطلاعات **پس از** $t_0$ یا خارج از مرز علت/فرآیند استفاده کند.

اگر مدل $\hat{y} = f(x)$ را آموزش دهید، نشت در واقع نقض این قرارداد است:
$$x = g(\text{اطلاعاتِ در دسترس در } t \le t_0).$$

**چرا EDA مهم است:** در EDA شما معنای واقعی ستون‌ها را می‌فهمید. بسیاری از خطاهای نشت به این دلیل اتفاق می‌افتند که ستون‌ها ظاهراً «معقول» هستند، اما در واقع از نتیجه، تصمیم‌های عملیاتی، یا اندازه‌گیری‌های پسینی مشتق شده‌اند.

### نشت در برابر «سیگنال قوی»
یک ویژگی می‌تواند به‌طور قانونی بسیار پیش‌بین باشد. بنابراین صرفاً «همبستگی زیاد» اثبات نشت نیست. تفاوت اصلی مربوط به *در دسترس بودن* و *فرآیند* است:

- **سیگنال معتبر:** در $t_0$ موجود است و از نظر علت/فرآیند در بالادستِ نتیجه قرار دارد.
- **سیگنالِ نشت:** پس از رخداد نتیجه (یا پس از تصمیم‌های کلیدی) تولید می‌شود یا مستقیماً برچسب را کُد می‌کند.

در این درس یاد می‌گیرید چگونه بررسی‌های **معنایی** (معنای ستون) و بررسی‌های **آماری** (قدرت پیش‌بینیِ غیرعادی + حساسیت به نوع Split) را ترکیب کنید تا تصمیم بگیرید با کدام حالت روبه‌رو هستید.

## ۲) یک طبقه‌بندی از نشت که می‌شود ممیزی‌اش کرد

نشت در چند شکل تکرارشونده ظاهر می‌شود. هنگام EDA، باید فعالانه دنبال این الگوها بگردید:

۱. **نشت برچسب (Target leakage):** یک ویژگی تابعی مستقیم از هدف (یا شبه‌هدف نزدیک) است.  
   مثال‌ها: `is_fraud_reviewed`، `chargeback_flag`، `final_grade`، `approved_amount` هنگام پیش‌بینی تأیید.

۲. **متغیرهای پس از وقوع نتیجه:** ویژگی‌هایی که بعد از مشخص‌شدن نتیجه ثبت می‌شوند، معمولاً در قالب یک گردش‌کار.  
   مثال‌ها: `resolution_code`، `time_to_close`، `refund_amount`، `response_status`.

۳. **نشت زمانی:** در Split داده، اطلاعات آینده نسبت به بازهٔ آزمون وارد آموزش می‌شود.  
   مثال: استفاده از داده‌های ۲۰۲۵ برای پیش‌بینی نتایج ۲۰۲۴.

۴. **نشت گروه/هویت:** همان موجودیت در آموزش و آزمون تکرار می‌شود و «هویت» را لو می‌دهد.  
   مثال: همان کاربر/بیمار/دستگاه/میزبان/فروشگاه/دانشجو در هر دو بخش.

۵. **نشت تکراری/نزدیک‌به‌تکراری:** ردیف‌های یکسان یا بسیار مشابه از مرز Split عبور می‌کنند.

۶. **نشت در پیش‌پردازش:** فیت‌کردن scaler / imputer / encoder / انتخاب ویژگی / PCA روی کل داده قبل از Split.

۷. **نشت در ارزیابی:** تیونینگ تکراری روی تست، یا استفاده از تست برای انتخاب ویژگی/هایپرپارامتر.

**در EDA معمولاً با یک نمودار نمی‌شود نشت را «اثبات» کرد.** به‌جایش یک پرونده می‌سازید با ترکیب:
- معنای ستون‌ها و دانش فرآیند، به‌علاوه
- سیگنال‌های کمیِ «مشکوک بودن»، به‌علاوه
- آزمایش‌های کنترل‌شده برای ایزوله‌کردن اثر ستون‌های مشکوک.

## ۳) جعبه‌ابزار: تشخیص‌های سریع و تکرارپذیر (کدی که می‌توانید دوباره استفاده کنید)

چند تابع کمکی می‌سازیم:

- `load_csv(path)`: بارگذاری مقاوم (نوع داده‌ها، مقدارهای گم‌شده).
- `summarize(df)`: خلاصهٔ EDA (نوع‌ها، گمشده‌ها، کاردینالیتی).
- `single_feature_screen(...)`: عملکرد Cross-Validated با یک ویژگی در هر بار.
- `split_sensitivity_test(...)`: مقایسهٔ عملکرد در Splitهای مختلف (تصادفی در برابر گروهی در برابر زمانی).
- `suspicious_name_scan(...)`: شکار ستون‌هایی که از روی نام شبیه شبه‌هدف/پسینی هستند.

هدف «کمال» نیست؛ هدف این است که سریع بفهمیم *کدام ستون‌ها نیاز به بررسی عمیق‌تر دارند*.

In [2]:
import os
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold, TimeSeriesSplit, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.base import clone

import matplotlib.pyplot as plt

np.random.seed(42)

def load_csv(path, **kwargs):
    """Load CSV using a repository-relative path."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}\nCheck the repository structure and your working directory.")
    # Try a couple of safe defaults for messy CSVs.
    defaults = dict(low_memory=False)
    defaults.update(kwargs)
    return pd.read_csv(path, **defaults)

def summarize(df, max_unique=20):
    """Basic EDA summary focused on leakage forensics."""
    out = []
    for c in df.columns:
        s = df[c]
        n = len(s)
        n_missing = int(s.isna().sum())
        miss_rate = n_missing / max(n, 1)
        nunique = int(s.nunique(dropna=True))
        dtype = str(s.dtype)
        example = s.dropna().iloc[0] if s.dropna().shape[0] else np.nan
        out.append({
            "column": c,
            "dtype": dtype,
            "missing_rate": round(miss_rate, 4),
            "nunique": nunique,
            "example_value": example if isinstance(example, (int, float, str)) else str(example),
            "low_cardinality": nunique <= max_unique
        })
    return pd.DataFrame(out).sort_values(["missing_rate", "nunique"], ascending=[False, False]).reset_index(drop=True)

def _infer_task(y):
    # Heuristic: small number of unique values -> classification
    n_unique = y.nunique(dropna=True)
    if n_unique <= 20 and (y.dtype == "object" or y.dtype.name.startswith("int") or y.dtype.name.startswith("bool")):
        return "classification"
    return "regression"

def _build_preprocessor(X):
    # Drop columns that are entirely missing (common in public CSVs and avoids imputer warnings)
    non_empty_cols = [c for c in X.columns if not X[c].isna().all()]
    if len(non_empty_cols) == 0:
        raise ValueError("All feature columns are entirely missing; cannot build a usable preprocessing pipeline.")
    Xn = X[non_empty_cols]

    num_cols = Xn.select_dtypes(include=["number"]).columns.tolist()
    cat_cols = [c for c in Xn.columns if c not in num_cols]

    numeric = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ])
    categorical = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", min_frequency=5)),
    ])

    pre = ColumnTransformer(
        transformers=[
            ("num", numeric, num_cols),
            ("cat", categorical, cat_cols),
        ],
        remainder="drop",
        sparse_threshold=0.3
    )
    return pre, num_cols, cat_cols

def single_feature_screen(df, target_col, task=None, max_cat_unique=50, cv_splits=5, random_state=42):
    """Cross-validated performance using one feature at a time. High scores can indicate leakage."""
    y = df[target_col]
    X_all = df.drop(columns=[target_col]).copy()

    if task is None:
        task = _infer_task(y)

    results = []
    # Ignore very high-cardinality text/id-like columns to keep the screen fast.
    for col in X_all.columns:
        s = X_all[col]
        nunique = s.nunique(dropna=True)
        if s.dtype == "object" and nunique > max_cat_unique:
            continue

        X = X_all[[col]].copy()

        pre, _, _ = _build_preprocessor(X)

        if task == "classification":
            # Use a robust linear baseline
            model = LogisticRegression(max_iter=2000, solver="lbfgs")
            pipe = Pipeline([("pre", pre), ("model", model)])
            cv = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=random_state)
            scores = cross_val_score(pipe, X, y, cv=cv, scoring="roc_auc")
            metric_name = "roc_auc"
        else:
            model = Ridge(alpha=1.0, random_state=random_state)
            pipe = Pipeline([("pre", pre), ("model", model)])
            cv = KFold(n_splits=cv_splits, shuffle=True, random_state=random_state)
            scores = cross_val_score(pipe, X, y, cv=cv, scoring="r2")
            metric_name = "r2"

        results.append({
            "feature": col,
            metric_name: float(np.mean(scores)),
            "std": float(np.std(scores)),
            "nunique": int(nunique),
            "dtype": str(s.dtype)
        })

    if not results:
        return pd.DataFrame(columns=["feature", "score", "std", "nunique", "dtype"])

    res = pd.DataFrame(results)
    score_col = "roc_auc" if task == "classification" else "r2"
    return res.sort_values(score_col, ascending=False).reset_index(drop=True)

def split_sensitivity_test(df, target_col, drop_cols=None, group_col=None, time_col=None, task=None, cv_splits=5, random_state=42):
    """Compare performance across splitting strategies. Large deltas can indicate leakage."""
    drop_cols = drop_cols or []
    y = df[target_col]
    X = df.drop(columns=[target_col] + drop_cols).copy()

    if task is None:
        task = _infer_task(y)

    pre, _, _ = _build_preprocessor(X)

    if task == "classification":
        model = LogisticRegression(max_iter=2000, solver="lbfgs")
        scoring = "roc_auc"
    else:
        model = Ridge(alpha=1.0, random_state=random_state)
        scoring = "r2"

    pipe = Pipeline([("pre", pre), ("model", model)])

    scores = {}

    # Random CV
    if task == "classification":
        cv_random = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=random_state)
    else:
        cv_random = KFold(n_splits=cv_splits, shuffle=True, random_state=random_state)
    scores["random_cv"] = float(np.mean(cross_val_score(pipe, X, y, cv=cv_random, scoring=scoring)))

    # Group CV (if provided)
    if group_col is not None:
        groups = df[group_col].values
        cv_group = GroupKFold(n_splits=min(cv_splits, len(np.unique(groups))))
        scores["group_cv"] = float(np.mean(cross_val_score(pipe, X, y, cv=cv_group, groups=groups, scoring=scoring)))

    # Time CV (if provided)
    if time_col is not None:
        # Sort by time and use expanding splits
        order = pd.to_datetime(df[time_col], errors="coerce").sort_values().index
        X_t = X.loc[order]
        y_t = y.loc[order]
        tscv = TimeSeriesSplit(n_splits=cv_splits)
        scores["time_cv"] = float(np.mean(cross_val_score(pipe, X_t, y_t, cv=tscv, scoring=scoring)))

    return scores

def suspicious_name_scan(columns):
    """Flag column names that often correlate with leakage in real projects."""
    patterns = [
        r"label", r"target", r"outcome", r"response", r"resolved", r"resolution", r"closed", r"approved",
        r"final", r"post", r"after", r"future", r"leak", r"score", r"status", r"decision", r"payout",
        r"refund", r"chargeback", r"dispute"
    ]
    rx = re.compile("|".join(patterns), flags=re.IGNORECASE)
    flagged = [c for c in columns if rx.search(c)]
    return flagged

## ۴) مطالعهٔ موردی A — طبقه‌بندی دیابت: ویژگی‌هایی که «بیش از حد خوب» هستند

دیتاست دیابت را بارگذاری می‌کنیم و یک خلاصهٔ EDA می‌گیریم. سپس یک خطای رایج نشت را **شبیه‌سازی** می‌کنیم: ویژگی‌ای که (مستقیم یا غیرمستقیم) از برچسب ساخته شده است.

این شبیه‌سازی عمدی است: در پروژه‌های واقعی، نشت معمولاً داخل ستون‌هایی پنهان می‌شود که ظاهراً «معقول» هستند (مثل کدها، فلگ‌های مشتق‌شده، اندازه‌گیری‌های پسینی). با ساختن یک ویژگی نشت‌دار، می‌توانیم مطمئن شویم ابزارهای تشخیصی واقعاً آن را شکار می‌کنند.

**هدف:** `classification` (Diabetic در برابر Non-Diabetic)

**پرسش‌های کلیدی برای کالبدشکافی**
- آیا از روی نام ستون‌ها چیزی مشکوک به نظر می‌رسد؟
- آیا یک ویژگی به‌تنهایی ROC-AUC بسیار بالا می‌دهد؟
- اگر ویژگی مشکوک را حذف کنیم، آیا افت عملکرد مطابق انتظارِ نشت رخ می‌دهد؟

### ۴.۱ بارگذاری و خلاصه‌سازی

ابتدا مقدارهای گم‌شده، نوع داده‌ها و کاردینالیتی را بررسی کنید. نشت معمولاً با این موارد هم‌راه است:
- ستون‌هایی شبیه ID (کاردینالیتی بالا)،
- فیلدهای وضعیت/گردش‌کار،
- ستون‌های زمانی یا «نتیجهٔ نهایی»،
- تجمیع‌های پس‌پردازش‌شده.

In [3]:
# --- Case study A: diabetes.csv ---
diabetes_path = "../../../Datasets/Classification/diabetes.csv"
df_diabetes = load_csv(diabetes_path)

display(df_diabetes.head())
summary_diabetes = summarize(df_diabetes)
display(summary_diabetes)

# Suspicious name scan
flagged = suspicious_name_scan(df_diabetes.columns)
print("Flagged by name patterns:", flagged)

# Ensure target is binary for ROC-AUC
df_diabetes = df_diabetes.copy()
df_diabetes["y"] = (df_diabetes["classification"].astype(str).str.lower() == "diabetic").astype(int)

# Inject a leaky proxy: label plus tiny noise (mimics a post-outcome proxy)
rng = np.random.default_rng(42)
df_diabetes["leaky_proxy"] = df_diabetes["y"] + rng.normal(0, 0.03, size=len(df_diabetes))

# Build a modeling frame
df_d = df_diabetes.drop(columns=["classification"]).copy()

# Single-feature screen
screen_d = single_feature_screen(df_d, target_col="y", task="classification", max_cat_unique=50, cv_splits=5)
display(screen_d.head(15))

# Split sensitivity with and without the leaky feature
scores_with = split_sensitivity_test(df_d, target_col="y", task="classification", cv_splits=5)
scores_without = split_sensitivity_test(df_d.drop(columns=["leaky_proxy"]), target_col="y", task="classification", cv_splits=5)
print("Scores WITH leaky feature:", scores_with)
print("Scores WITHOUT leaky feature:", scores_without)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,classification
0,6,148,72,35,0,33.6,0.627,50,Diabetic
1,1,85,66,29,0,26.6,0.351,31,Non-Diabetic
2,8,183,64,0,0,23.3,0.672,32,Diabetic
3,1,89,66,23,94,28.1,0.167,21,Non-Diabetic
4,0,137,40,35,168,43.1,2.288,33,Diabetic


,column,dtype,missing_rate,nunique,example_value,low_cardinality
0,DiabetesPedigreeFunction,float64,0.0,517,0.627,False
1,BMI,float64,0.0,248,33.6,False
2,Insulin,int64,0.0,186,0,False
3,Glucose,int64,0.0,136,148,False
4,Age,int64,0.0,52,50,False
5,SkinThickness,int64,0.0,51,35,False
6,BloodPressure,int64,0.0,47,72,False
7,Pregnancies,int64,0.0,17,6,True
8,classification,object,0.0,2,Diabetic,True


Flagged by name patterns: []


,feature,roc_auc,std,nunique,dtype
0,leaky_proxy,1.000000,0.000000,768,float64
1,Glucose,0.787365,0.026682,136,int64
2,BMI,0.688052,0.032472,248,float64
3,Age,0.685324,0.037894,52,int64
4,Pregnancies,0.619985,0.037818,17,int64
5,DiabetesPedigreeFunction,0.603836,0.052310,517,float64
6,BloodPressure,0.587046,0.037250,47,int64
7,SkinThickness,0.553768,0.049009,51,int64
8,Insulin,0.536359,0.070975,186,int64


Scores WITH leaky feature: {'random_cv': 1.0}
Scores WITHOUT leaky feature: {'random_cv': 0.8303878406708595}


### ۴.۲ ساختن یک شبه‌هدف نشت‌دار و شناسایی آن

یک ویژگی مصنوعی به نام `leaky_proxy` می‌سازیم که برچسب را با کمی نویز کُد می‌کند.
در یک دیتاست واقعی، این می‌تواند متناظر با «کد درمان پس از تشخیص» یا «کلاس صورتحساب» باشد که عملاً هدف را لو می‌دهد.

انتظار داریم غربال تک‌ویژگی این ستون را در رتبه‌های بالای لیست قرار دهد.

### ۴.۳ تفسیر نتایج (چطور استدلال کنیم، نه فقط محاسبه)

اگر در غربال تک‌ویژگی به ستون‌هایی برسید که **ROC-AUC نزدیک ۱.۰** دارند، این یک هشدار جدی است:

- ممکن است ستون مستقیماً برچسب را کُد کرده باشد.
- ممکن است پیامد تعیین‌کنندهٔ برچسب باشد (پس از وقوع نتیجه).
- ممکن است یک شناسه باشد که به‌طور غیرمستقیم هدف را لو می‌دهد (به علت آرتیفکت‌های جمع‌آوری داده).

گام بعدی «حذف فوری» نیست؛ بلکه باید بپرسید:

۱) **این ویژگی چه زمانی در دسترس است؟** ($t_0$ را تعریف کنید.)  
۲) **چه کسی/چه سیستمی آن را تولید می‌کند؟** (گردش‌کار عملیاتی؟ بازبینی انسانی؟ صورتحساب؟)  
۳) **آیا بدون دانستن هدف قابل محاسبه است؟** (اگر نه، نشت است.)

در پروژه‌های واقعی پاسخ این پرسش‌ها با متخصصان دامنه، ردیابی داده (data lineage)، و لاگ‌های سیستم داده می‌شود. در EDA وظیفهٔ شما «بیرون کشیدن کاندیداهای مشکوک» است.

## ۵) مطالعهٔ موردی B — رگرسیون قیمت خانه: نشت از طریق ویژگی‌های «مشتق از هدف»

در رگرسیون، نشت اغلب وقتی رخ می‌دهد که کسی ویژگی‌هایی به ظاهر «کمک‌کننده» بسازد که در واقع بازآرایی جبریِ هدف هستند. مثال:

- پیش‌بینی `Price` و اضافه‌کردن ویژگی `Price_per_sqft = Price / SqFt`  
  این یعنی پاسخ را به مدل داده‌اید.

ما:
- `house-prices.csv` را بارگذاری می‌کنیم،
- یک مدل پایه برای تخمین عملکرد می‌سازیم،
- دو ویژگی مهندسی‌شدهٔ نشت‌دار اضافه می‌کنیم،
- و نشان می‌دهیم این نشت هم در غربال تک‌ویژگی و هم در افزایش غیرواقعی عملکرد قابل مشاهده است.

**هدف:** `Price`

**قاعدهٔ هشدار:** اگر ویژگی‌ای از نظر ریاضی به هدف وابسته باشد، یعنی $x_j = h(y, \cdot)$، آن ویژگی برای پیش‌بینی $y$ در زمان امتیازدهی معتبر نیست.

In [4]:
# --- Case study B: house-prices.csv ---
house_path = "../../../Datasets/Regression/house-prices.csv"
df_house = load_csv(house_path)

display(df_house.head())
display(summarize(df_house))

# Baseline: predict Price
df_h = df_house.copy()

# Inject leaky engineered features
df_h["price_per_sqft_leak"] = df_h["Price"] / df_h["SqFt"].replace(0, np.nan)
df_h["log_price_leak"] = np.log1p(df_h["Price"])

# Screen single features for regression
screen_h = single_feature_screen(df_h, target_col="Price", task="regression", max_cat_unique=50, cv_splits=5)
display(screen_h.head(15))

# Compare performance with and without leaky columns
scores_with = split_sensitivity_test(df_h, target_col="Price", task="regression", cv_splits=5)
scores_without = split_sensitivity_test(df_h.drop(columns=["price_per_sqft_leak", "log_price_leak"]), target_col="Price", task="regression", cv_splits=5)

print("Regression scores WITH leaky features:", scores_with)
print("Regression scores WITHOUT leaky features:", scores_without)

,Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood
0,1,114300,1790,2,2,2,No,East
1,2,114200,2030,4,2,3,No,East
2,3,114800,1740,3,2,1,No,East
3,4,94700,1980,3,2,3,No,East
4,5,119800,2130,3,3,3,No,East


,column,dtype,missing_rate,nunique,example_value,low_cardinality
0,Home,int64,0.0,128,1,False
1,Price,int64,0.0,123,114300,False
2,SqFt,int64,0.0,61,1790,False
3,Offers,int64,0.0,6,2,True
4,Bedrooms,int64,0.0,4,2,True
5,Bathrooms,int64,0.0,3,2,True
6,Neighborhood,object,0.0,3,East,True
7,Brick,object,0.0,2,No,True


,feature,r2,std,nunique,dtype
0,log_price_leak,0.977711,0.007256,123,float64
1,price_per_sqft_leak,0.683828,0.083864,128,float64
2,Neighborhood,0.450231,0.126492,3,object
3,SqFt,0.228876,0.100532,61,int64
4,Bedrooms,0.227696,0.182803,4,int64
5,Bathrooms,0.120441,0.232334,3,int64
6,Brick,0.092262,0.104749,2,object
7,Offers,-0.016599,0.113085,6,int64
8,Home,-0.104937,0.113642,128,int64


Regression scores WITH leaky features: {'random_cv': 0.9895991548830227}
Regression scores WITHOUT leaky features: {'random_cv': 0.8275952075387714}


## ۶) مطالعهٔ موردی C — دادهٔ Listings: ستون‌های پسینی و حساسیت به Split

دیتاست‌های واقعی اغلب ستون‌هایی دارند که فقط **بعد از** شکل‌گیری یک تاریخچهٔ عملیاتی مشخص می‌شوند.
در دادهٔ لیستینگ‌ها، ستون‌های شبه‌پسینی رایج عبارت‌اند از:
- `number_of_reviews`, `reviews_per_month`, `last_review` (وابسته به رزرو/بازبینی‌های آینده)
- `availability_365` (وابسته به اشغال آیندهٔ تقویم)

فرض کنید هدف شما این است: «قیمت لیستینگ را در زمان ایجاد لیستینگ برآورد کن.»
در این صورت ستون‌های مرتبط با review ممکن است قرارداد $t_0$ را نقض کنند.

علاوه بر این، لیستینگ‌ها گروه طبیعی دارند (مثل `host_id`). اگر یک میزبان هم در آموزش و هم در آزمون باشد، CV تصادفی می‌تواند بیش از حد خوش‌بینانه شود.

ما نشان می‌دهیم:
- اسکن مشکوک از روی نام ستون‌ها،
- تفاوت split گروهی و تصادفی،
- split زمانی (وقتی ستون زمانی قابل استفاده وجود داشته باشد).

In [5]:
# --- Case study C: listings.csv ---
listings_path = "../../../Datasets/Regression/listings.csv"
df_listings = load_csv(listings_path)

display(df_listings.head())
display(summarize(df_listings))

# Define a simple target to keep the tutorial self-contained:
# Predict whether a listing is "high price" (binary classification).
# (We ignore rows with missing price.)
dfl = df_listings.copy()
dfl["price"] = pd.to_numeric(dfl["price"], errors="coerce")
dfl = dfl.dropna(subset=["price"]).reset_index(drop=True)

# Median-based label (robust and easy to explain)
median_price = float(dfl["price"].median())
dfl["y_high_price"] = (dfl["price"] >= median_price).astype(int)

# Columns that may be post-outcome relative to listing creation time
suspected_post_outcome = ["number_of_reviews", "last_review", "reviews_per_month", "number_of_reviews_ltm", "availability_365"]
present_suspects = [c for c in suspected_post_outcome if c in dfl.columns]
print("Potential post-outcome columns present:", present_suspects)

# Quick name scan
flagged = suspicious_name_scan(dfl.columns)
print("Flagged by name patterns:", flagged)

# Compare random CV vs group CV (host_id)
drop_cols = []
for c in ["id", "name", "host_name", "license"]:
    if c in dfl.columns:
        drop_cols.append(c)

# Include suspects first, then compare when we drop them
scores_random_vs_group = split_sensitivity_test(
    dfl.drop(columns=drop_cols),
    target_col="y_high_price",
    task="classification",
    group_col="host_id" if "host_id" in dfl.columns else None,
    time_col="last_review" if "last_review" in dfl.columns else None,
    cv_splits=5
)
print("Scores (random/group/time if available):", scores_random_vs_group)

# Now drop suspected post-outcome features and re-test
cols_to_drop = [c for c in present_suspects if c in dfl.columns]
dfl_noleak = dfl.drop(columns=cols_to_drop + drop_cols, errors="ignore")
scores_no_post = split_sensitivity_test(
    dfl_noleak,
    target_col="y_high_price",
    task="classification",
    group_col="host_id" if "host_id" in dfl_noleak.columns else None,
    time_col="last_review" if "last_review" in dfl_noleak.columns else None,
    cv_splits=5
)
print("Scores after dropping suspected post-outcome columns:", scores_no_post)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,13913,Holiday London DB Room Let-on going,54730,Alina,NaN,Islington,51.56861,-0.11270,Private room,57.0,1,51,2025-02-09,0.29,3,344,10,NaN
1,15400,Bright Chelsea Apartment. Chelsea!,60302,Philippa,NaN,Kensington and Chelsea,51.48780,-0.16813,Entire home/apt,NaN,4,96,2024-04-28,0.52,1,11,2,NaN
2,17402,Very Central Modern 3-Bed/2 Bath By Oxford St W1,67564,Liz,NaN,Westminster,51.52195,-0.14094,Entire home/apt,510.0,3,56,2024-02-19,0.33,5,293,0,NaN
3,24328,Battersea live/work artist house,41759,Joe,NaN,Wandsworth,51.47072,-0.16266,Entire home/apt,213.0,90,94,2022-07-19,0.54,1,194,0,NaN
4,31036,Bright compact 1 Bedroom Apartment Brick Lane,133271,Hendryks,NaN,Tower Hamlets,51.52425,-0.06997,Entire home/apt,100.0,2,126,2025-02-20,0.70,8,353,3,NaN


,column,dtype,missing_rate,nunique,example_value,low_cardinality
0,neighbourhood_group,float64,1.0000,0,NaN,True
1,license,float64,1.0000,0,NaN,True
2,price,float64,0.3619,1239,57.0,False
3,last_review,object,0.2564,3444,2025-02-09,False
4,reviews_per_month,float64,0.2564,868,0.29,False
5,host_name,object,0.0006,16416,Alina,False
6,id,int64,0.0000,94559,13913,False
7,name,object,0.0000,90942,Holiday London DB Room Let-on going,False
8,longitude,float64,0.0000,61792,-0.1127,False
9,host_id,int64,0.0000,55395,54730,False


Potential post-outcome columns present: ['number_of_reviews', 'last_review', 'reviews_per_month', 'number_of_reviews_ltm', 'availability_365']
Flagged by name patterns: []
Scores (random/group/time if available): {'random_cv': 0.9994904216839016, 'group_cv': 0.9994761209249441, 'time_cv': 0.9973845805463218}
Scores after dropping suspected post-outcome columns: {'random_cv': 0.9997040883162702, 'group_cv': 0.9996974451218641}


### ۶.۱ تفسیر حساسیت به نوع Split

وقتی **CV تصادفی** خیلی بهتر از **CV گروهی** است، یک توضیح رایج نشت هویتی است:

- مدل الگوهای خاصِ میزبان (یا شبه‌هویت‌های محله‌ای) را یاد می‌گیرد.
- در فولد تست همان میزبان‌ها هم وجود دارند و مسئله غیرواقعی ساده می‌شود.

وقتی **CV زمانی** بدتر از CV تصادفی است، توضیح رایج نشت زمانی و تغییر توزیع است:

- اگر زمان را نادیده بگیرید، الگوهای آینده وارد آموزش می‌شوند،
- یا فرآیند تولید داده در طول زمان تغییر می‌کند (drift).

حساسیت به Split به‌تنهایی «اثبات» نشت نیست. اما به شما می‌گوید **کجا باید عمیق‌تر بررسی کنید**:
- کدام ستون‌ها عملاً «شبه-ID» هستند؟
- کدام ستون‌ها نیازمند سیاست «as-of time» هستند؟
- آیا مدل به جای رابطهٔ قابل تعمیم، در حال یادگیری یک آرتیفکت cohort است؟

## ۷) مطالعهٔ موردی D — شکایات مصرف‌کننده: ستون‌های گردش‌کارِ پس از وقوع نتیجه

دیتاست شکایات برای تمرین نشت بسیار مناسب است چون هم شامل:
- اطلاعات ورودی/ابتدایی (آن‌چه هنگام دریافت شکایت می‌دانیم)، و هم
- خروجی‌های گردش‌کار (آن‌چه بعد از پردازش شکایت رخ می‌دهد)
است.

برای شفاف‌کردن مرز «as-of time»، یک هدف واقعی تعریف می‌کنیم:

**هدف پیش‌بینی:** پیش‌بینی کنیم آیا مصرف‌کننده نتیجه را مورد اختلاف قرار می‌دهد (`Consumer Disputed`) *در زمان دریافت شکایت*.

در آن لحظه، بسیاری از ستون‌ها هنوز **در دسترس نیستند**:
- `Company Response to Consumer`
- `Timely Response`
- `Date Sent to Company`
- `Company Public Response`
- و گاهی کدهای وضعیت داخلی

استفاده از این ستون‌ها می‌تواند نشت جدی ایجاد کند: شما از فرآیند حل‌وفصل برای پیش‌بینی اختلاف استفاده کرده‌اید.

ما:
- دیتاست را بارگذاری می‌کنیم،
- دو مجموعه ویژگی می‌سازیم (فقط intake در برابر intake+workflow)،
- عملکرد را مقایسه می‌کنیم،
- و ستون‌هایی که بیشترین تورم عملکرد را ایجاد می‌کنند مشخص می‌کنیم.

In [6]:
# --- Case study D: ConsumerComplaints.csv ---
complaints_path = "../../../Datasets/Clustering/ConsumerComplaints.csv"
df_cc = load_csv(complaints_path)

display(df_cc.head())
display(summarize(df_cc))

# Clean up target: Consumer Disputed (Yes/No)
dfc = df_cc.copy()
if "Consumer Disputed" not in dfc.columns:
    raise ValueError("Expected column 'Consumer Disputed' not found in ConsumerComplaints.csv")

# Drop rows where target is missing
dfc = dfc.dropna(subset=["Consumer Disputed"]).reset_index(drop=True)

dfc["y_disputed"] = (dfc["Consumer Disputed"].astype(str).str.strip().str.lower() == "yes").astype(int)

# Define intake-time columns (best-effort, based on typical intake fields)
intake_cols = []
for c in ["Date Received", "Product Name", "Sub Product", "Issue", "Sub Issue", "Company", "State Name", "Zip Code", "Tags", "Submitted via"]:
    if c in dfc.columns:
        intake_cols.append(c)

# Define workflow/post-outcome-ish columns (likely not known at intake time)
workflow_cols = []
for c in ["Date Sent to Company", "Company Response to Consumer", "Company Public Response", "Timely Response", "Complaint ID"]:
    if c in dfc.columns:
        workflow_cols.append(c)

print("Intake columns:", intake_cols)
print("Workflow columns:", workflow_cols)

# Build two datasets:
# 1) intake-only
df_intake = dfc[intake_cols + ["y_disputed"]].copy()

# 2) intake + workflow
df_full = dfc[intake_cols + workflow_cols + ["y_disputed"]].copy()

# Evaluate split sensitivity (random CV) as a quick proxy
scores_intake = split_sensitivity_test(df_intake, target_col="y_disputed", task="classification", cv_splits=5)
scores_full = split_sensitivity_test(df_full, target_col="y_disputed", task="classification", cv_splits=5)

print("Intake-only scores:", scores_intake)
print("Intake+workflow scores:", scores_full)

# Single-feature screen on the full set to see which columns are suspiciously predictive
screen_cc = single_feature_screen(df_full, target_col="y_disputed", task="classification", max_cat_unique=50, cv_splits=5)
display(screen_cc.head(20))

,Date Received,Product Name,Sub Product,Issue,Sub Issue,Consumer Complaint Narrative,Company Public Response,Company,State Name,Zip Code,Tags,Consumer Consent Provided,Submitted via,Date Sent to Company,Company Response to Consumer,Timely Response,Consumer Disputed,Complaint ID
0,2013-07-29,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,NaN,NaN,Wells Fargo & Company,VA,24540,NaN,NaN,Phone,2013-07-30,Closed with explanation,Yes,No,468882
1,2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,NaN,NaN,NaN,Wells Fargo & Company,CA,95992,Older American,NaN,Web,2013-07-31,Closed with explanation,Yes,No,468889
2,2013-07-29,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,Santander Bank US,NY,10065,NaN,NaN,Fax,2013-07-31,Closed,Yes,No,468879
3,2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,NaN,NaN,NaN,Wells Fargo & Company,GA,30084,NaN,NaN,Web,2013-07-30,Closed with explanation,Yes,No,468949
4,2013-07-29,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Franklin Credit Management,CT,6106,NaN,NaN,Web,2013-07-30,Closed with explanation,Yes,No,475823


,column,dtype,missing_rate,nunique,example_value,low_cardinality
0,Company Public Response,object,0.9626,9,Company chooses not to provide a public response,True
1,Consumer Complaint Narrative,object,0.9601,2592,Received Capital One charge card offer XXXX. A...,False
2,Consumer Consent Provided,object,0.9273,3,Consent provided,True
3,Tags,object,0.8522,3,Older American,True
4,Sub Issue,object,0.5319,67,Debt is not mine,False
5,Sub Product,object,0.2834,45,Vehicle loan,False
6,Consumer Disputed,object,0.0169,2,No,True
7,Zip Code,object,0.0073,14851,24540,False
8,State Name,object,0.0073,59,VA,False
9,Complaint ID,int64,0.0000,65499,468882,False


Intake columns: ['Date Received', 'Product Name', 'Sub Product', 'Issue', 'Sub Issue', 'Company', 'State Name', 'Zip Code', 'Tags', 'Submitted via']
Workflow columns: ['Date Sent to Company', 'Company Response to Consumer', 'Company Public Response', 'Timely Response', 'Complaint ID']
Intake-only scores: {'random_cv': 0.5900515995185385}
Intake+workflow scores: {'random_cv': 0.6054917298073457}


,feature,roc_auc,std,nunique,dtype
0,Company Response to Consumer,0.553540,0.003773,4,object
1,Product Name,0.544824,0.009093,11,object
2,Sub Product,0.543912,0.006271,45,object
3,Submitted via,0.543063,0.001527,6,object
4,Complaint ID,0.505308,0.004983,64391,int64
5,Company Public Response,0.501944,0.000621,9,object
6,Tags,0.501036,0.001619,3,object
7,Timely Response,0.500367,0.001001,2,object


### ۷.۱ تفکر «as-of time»: انضباطی که نشت را مهار می‌کند

وقتی قرارداد **as-of time** را صریح مستندسازی کنید، پیشگیری از نشت قابل کنترل می‌شود:

- $t_0$ برای این پیش‌بینی دقیقاً چیست؟
- کدام سیستم‌های منبع مجاز هستند؟
- چه رویدادهایی باید قبل از ایجاد یک ویژگی رخ داده باشند؟
- چه Joinها/تجمیع‌هایی مجاز هستند؟

در یک feature store واقعی، این موضوع با joinهای رویداد-زمانی و «درستیِ نقطه-در-زمان» enforce می‌شود.
در نوت‌بوک، شما با این کارها آن را enforce می‌کنید:
- فرضِ در دسترس بودن را می‌نویسید،
- مجموعه ویژگی را به ستون‌های سازگار محدود می‌کنید،
- و با split زمانی/گروهی اعتبارسنجی می‌کنید.

یک قاعدهٔ مفید:

- اگر ویژگی توسط یک فرآیند پایین‌دست تولید می‌شود که با هدف تحریک می‌شود (یا با اقداماتی که بعد از مشاهدهٔ هدف انجام می‌شوند)، احتمالاً پسینی و در $t_0$ نامعتبر است.

به همین دلیل ستون‌هایی مانند «resolution»، «closed»، «timely response»، و «final status» معمولاً عامل نشت هستند.

## ۸) فراتر از مبانی: الگوهای قوی‌تر برای کالبدشکافی نشت

غربال‌های بالا عمداً سبک هستند. در کارهای حساس‌تر باید بررسی‌های عمیق‌تر اضافه کنید.

### ۸.۱ آزمون تورم عملکردِ شرطی

علامت اصلی نشت این است که مدل خوب عمل می‌کند چون می‌تواند به اطلاعات ممنوع «نگاه کند».
یک آزمون عملی:

۱) یک مدل پایه روی مجموعه ویژگی «ایمن» آموزش دهید.  
۲) یک ستون مشکوک را اضافه کنید.  
۳) با همان پروتکل ارزیابی دوباره آموزش دهید.  
۴) اگر عملکرد جهش بزرگی داشت و ستون در $t_0$ در دسترس نیست، آن را نشت در نظر بگیرید.

### ۸.۲ پایداری نسبت به نوع Split

مقایسه کنید:
- K-fold تصادفی،
- K-fold گروهی (آگاه از موجودیت)،
- split زمانی (در صورت امکان).

اگر ویژگی واقعاً قابل تعمیم باشد، عملکرد نباید تحت split واقع‌بینانه به‌شدت فروبپاشد.
فروپاشی لزوماً نشت را ثابت نمی‌کند (تغییر توزیع هم می‌تواند این کار را بکند)، اما سیگنال قوی برای تحقیق است.

### ۸.۳ کشف تکراری/نزدیک‌به‌تکراری

در EDA بررسی کنید:
- ردیف‌های کاملاً تکراری،
- تکرار روی شناسه‌های کلیدی،
- نزدیک‌به‌تکراری‌ها (شباهت زیاد روی چندین فیلد).

نشت از طریق تکراری‌ها در داده‌های scrape یا merge شده رایج است.

### ۸.۴ ممیزی نشت در پیش‌پردازش

اگر هر آماری را با کل داده محاسبه کنید (میانگین، واریانس، PCA، target encoding، انتخاب ویژگی)، ریسک نشت دارید.
الگوی امن همیشه این است:

- اول split،
- فیت پیش‌پردازش فقط روی آموزش،
- اعمال روی اعتبارسنجی/تست با `transform`.

در scikit-learn وقتی پیش‌پردازش داخل `Pipeline` باشد و Cross-Validation را روی همان پایپلاین انجام دهید، این الگو به‌صورت طبیعی enforce می‌شود.

## ۸.۵ نشت در Target Encoding و تجمیع‌ها (تلهٔ رایج در EDA)

بعضی از ظریف‌ترین خطاهای نشت *همان هنگام EDA* ساخته می‌شوند—به‌خصوص وقتی خلاصه‌های «کمک‌کننده» از هدف می‌سازید.

### Target Encoding: چرا به‌سادگی نشت ایجاد می‌کند؟

Target encoding یک دسته را با آماری از هدف جایگزین می‌کند، معمولاً:

$$\mathrm{TE}(c) = \mathbb{E}[y \mid x=c].$$

اگر $\mathrm{TE}(c)$ را با کل داده محاسبه کنید و سپس Cross-Validation انجام دهید، ردیف‌های اعتبارسنجی هر فولد در محاسبهٔ encoding دخالت کرده‌اند. این یعنی نشت از مسیر پیش‌پردازش.

رویکرد درست **Cross-Fitting** است:
- برای هر فولد، encoding را فقط با بخش آموزش همان فولد بسازید،
- و سپس روی بخش اعتبارسنجی همان فولد اعمال کنید.

در ادامه، تورم عملکردِ target encoding ساده‌لوحانه را در برابر target encoding با cross-fit روی دیتاست دیابت نشان می‌دهیم.

حتی اگر قصد استفادهٔ عملی از target encoding را ندارید، این مثال یک پیام کلی دارد:

- هر ویژگی‌ای که از هدف مشتق می‌شود باید طوری محاسبه شود که مرز Split را رعایت کند.

### ۸.۶ درستیِ نقطه-در-زمان برای تجمیع‌های زمانی

الگوی نشت رایج دیگر این است که تجمیع‌هایی بسازید که ناخواسته اطلاعات آینده را وارد می‌کنند. فرض کنید رویدادهایی با $(e_i, t_i)$ داریم:

- می‌خواهید در $t_0$ پیش‌بینی کنید،
- یک ویژگی مثل «جمع کل هزینهٔ مشتری» را با جمع تراکنش‌ها می‌سازید،
- اگر تراکنش‌های با $t > t_0$ هم وارد جمع شوند، ویژگی قرارداد در دسترس بودن را نقض می‌کند.

از نظر نمادین، تجمیع امن چنین است:
$$\mathrm{agg}(t_0) = \sum_{i: t_i \le t_0} v_i,$$
نه
$$\sum_{i} v_i.$$

در دیتاست‌های جدولی بدون لاگ رویداد هم این نشت رخ می‌دهد وقتی:
- میانگین‌ها را به ازای گروه با کل داده حساب می‌کنید،
- نرخ‌های تاریخی را با ردیف‌های آینده محاسبه می‌کنید،
- آمارهای rolling را می‌سازید ولی ترتیب زمانی را رعایت نمی‌کنید.

در دیتاست لیستینگ‌ها، یک تجمیع گروهیِ عمداً نشت‌دار می‌سازیم که از *برچسب* برای ساخت آمارهٔ سطح میزبان استفاده می‌کند، سپس نشان می‌دهیم نسخهٔ ایمن‌تر را چگونه با cross-fitting بسازیم.

In [7]:
# --- Demonstration: naïve target encoding vs cross-fitted target encoding ---

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

def naive_target_encode(series, y):
    """Compute target mean per category using the full dataset (THIS LEAKS under CV)."""
    df_tmp = pd.DataFrame({"x": series.astype(str), "y": y})
    means = df_tmp.groupby("x")["y"].mean()
    return series.astype(str).map(means).astype(float)

def cv_target_encode(series, y, cv):
    """Cross-fitted target encoding: encodings computed on train folds only."""
    series = series.astype(str)
    y = np.asarray(y)
    enc = np.zeros(len(series), dtype=float)

    for tr_idx, va_idx in cv.split(series, y):
        x_tr = series.iloc[tr_idx]
        y_tr = y[tr_idx]
        means = pd.DataFrame({"x": x_tr, "y": y_tr}).groupby("x")["y"].mean()
        # For unseen categories in validation, use the global mean of the training fold
        global_mean = float(np.mean(y_tr))
        enc_va = series.iloc[va_idx].map(means).fillna(global_mean).astype(float)
        enc[va_idx] = enc_va.values

    return enc

# Use diabetes dataset with a simple categorical binning of Age
d = df_diabetes.copy()
d["age_bin"] = pd.cut(d["Age"], bins=[0, 25, 35, 45, 55, 100], include_lowest=True).astype(str)

y = d["y"].values
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Naïve target encoding (leaky under CV)
te_naive = naive_target_encode(d["age_bin"], y)

# Cross-fitted target encoding (respects CV boundary)
te_cv = cv_target_encode(d["age_bin"], y, cv)

# Evaluate AUC using the encoded value as a score (no model needed)
auc_naive = []
auc_cv = []
for tr_idx, va_idx in cv.split(d, y):
    auc_naive.append(roc_auc_score(y[va_idx], te_naive.iloc[va_idx]))
    auc_cv.append(roc_auc_score(y[va_idx], te_cv[va_idx]))

print("Naïve TE mean AUC (leaky):", float(np.mean(auc_naive)))
print("Cross-fitted TE mean AUC:", float(np.mean(auc_cv)))

# --- Demonstration: leaky group aggregate on listings ---

if "host_id" in dfl.columns:
    temp = dfl.copy()

    # Deliberately leaky: uses the label y_high_price to compute host-level rate using ALL rows
    host_rate_leaky = temp.groupby("host_id")["y_high_price"].mean()
    temp["host_highprice_rate_leaky"] = temp["host_id"].map(host_rate_leaky)

    # Cross-fitted host rate: compute host rates on train fold only
    def cv_group_rate(df, group_col, y_col, cv):
        y = df[y_col].values
        out = np.zeros(len(df), dtype=float)
        for tr_idx, va_idx in cv.split(df, y):
            tr = df.iloc[tr_idx]
            va = df.iloc[va_idx]
            rates = tr.groupby(group_col)[y_col].mean()
            global_mean = float(tr[y_col].mean())
            out[va_idx] = va[group_col].map(rates).fillna(global_mean).astype(float).values
        return out

    cv2 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    temp["host_highprice_rate_cv"] = cv_group_rate(temp, "host_id", "y_high_price", cv2)

    # Evaluate how predictive these two versions are (AUC as a proxy)
    auc_leaky = []
    auc_safe = []
    y2 = temp["y_high_price"].values
    for tr_idx, va_idx in cv2.split(temp, y2):
        auc_leaky.append(roc_auc_score(y2[va_idx], temp.loc[va_idx, "host_highprice_rate_leaky"]))
        auc_safe.append(roc_auc_score(y2[va_idx], temp.loc[va_idx, "host_highprice_rate_cv"]))

    print("Leaky host-rate mean AUC:", float(np.mean(auc_leaky)))
    print("Cross-fitted host-rate mean AUC:", float(np.mean(auc_safe)))
else:
    print("host_id column not found in listings sample; skipping host-rate demo.")

Naïve TE mean AUC (leaky): 0.6847631027253668
Cross-fitted TE mean AUC: 0.6815408805031445
Leaky host-rate mean AUC: 0.9843660815848916
Cross-fitted host-rate mean AUC: 0.7843591638684562


In [8]:
# --- Duplicate and near-duplicate checks (generic utilities) ---

def duplicate_report(df, subset=None):
    if subset is None:
        dup = df.duplicated(keep=False)
    else:
        dup = df.duplicated(subset=subset, keep=False)
    n_dup = int(dup.sum())
    print(f"Duplicate rows (subset={subset}): {n_dup}")
    if n_dup > 0:
        display(df.loc[dup].head(20))

# Examples: run on each dataset
duplicate_report(df_diabetes)
duplicate_report(df_house)
duplicate_report(df_listings, subset=["host_id"] if "host_id" in df_listings.columns else None)

Duplicate rows (subset=None): 0
Duplicate rows (subset=None): 0
Duplicate rows (subset=['host_id']): 49863


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,13913,Holiday London DB Room Let-on going,54730,Alina,NaN,Islington,51.56861,-0.112700,Private room,57.0,1,51,2025-02-09,0.29,3,344,10,NaN
2,17402,Very Central Modern 3-Bed/2 Bath By Oxford St W1,67564,Liz,NaN,Westminster,51.52195,-0.140940,Entire home/apt,510.0,3,56,2024-02-19,0.33,5,293,0,NaN
4,31036,Bright compact 1 Bedroom Apartment Brick Lane,133271,Hendryks,NaN,Tower Hamlets,51.52425,-0.069970,Entire home/apt,100.0,2,126,2025-02-20,0.70,8,353,3,NaN
5,33332,Beautiful Ensuite Richmond-upon-Thames borough,144444,Chi-Chi,NaN,Richmond upon Thames,51.46410,-0.324980,Private room,133.0,2,19,2022-08-01,0.11,2,365,0,NaN
7,36660,You are GUARANTEED to love this,157884,Agri & Roger,NaN,Haringey,51.58478,-0.160570,Private room,74.0,2,711,2025-02-26,4.03,2,193,51,NaN
8,38610,CHARMING FAMILY HOME,165579,Elisa & Dom,NaN,Hammersmith and Fulham,51.50701,-0.233620,Entire home/apt,NaN,91,42,2023-08-27,0.27,2,76,0,NaN
9,38950,Room 1 Large Double Bedroom - front ground floor,167107,Paul,NaN,Haringey,51.58684,-0.086320,Private room,52.0,1,1,2021-12-12,0.03,2,89,0,NaN
14,41712,"Room with a view, shared flat, central Bankside",182322,Nina,NaN,Southwark,51.50191,-0.101998,Private room,NaN,2,131,2024-10-14,0.77,2,179,7,NaN
16,42010,You Will Save Money Here,157884,Agri & Roger,NaN,Barnet,51.58590,-0.164340,Private room,55.0,2,613,2025-03-13,3.48,2,175,50,NaN
18,43129,Quiet Comfortable Room in Fulham,188138,Sylvan,NaN,Hammersmith and Fulham,51.48164,-0.210820,Private room,66.0,3,248,2025-01-02,1.75,3,242,9,NaN


## ۹) چک‌لیست عملی کالبدشکافی نشت (هر بار استفاده کنید)

این چک‌لیست را هنگام EDA و قبل از مدل‌سازی اجرا کنید:

**A. قرارداد پیش‌بینی را تعریف کنید**
- $y$ دقیقاً چیست؟
- $t_0$ (زمان پیش‌بینی) چیست؟
- چه منابعی در $t_0$ در دسترس‌اند؟

**B. معنای ستون‌ها**
- فیلدهای وضعیت/گردش‌کار، نتایج «نهایی»، و خلاصه‌های پسینی را مشخص کنید.
- IDها و کلیدها را مشخص کنید (کاردینالیتی بالا).
- ستون‌های زمانی و معنای رویدادشان را مشخص کنید.

**C. غربال‌های سریع مشکوک بودن**
- اسکن نام‌ها (`suspicious_name_scan`).
- غربال عملکرد تک‌ویژگی (`single_feature_screen`).

**D. آزمایش‌های کنترل‌شده**
- ستون‌های مشکوک را حذف کنید و افت عملکرد را بررسی کنید.
- split تصادفی/گروهی/زمانی را مقایسه کنید (`split_sensitivity_test`).

**E. بهداشت پیش‌پردازش**
- همهٔ transformها داخل pipeline باشند.
- هیچ‌گاه پیش‌پردازش را قبل از split روی کل داده فیت نکنید.

**F. مستندسازی**
- ثبت کنید چه ستون‌هایی حذف شدند و چرا.
- فرض‌های قرارداد as-of را ثبت کنید.

اگر این کار را به عادت تبدیل کنید، نشت تبدیل به چیزی می‌شود که زود تشخیص می‌دهید، نه فاجعه‌ای که آخر کار مجبور به دیباگش شوید.

## ۱۰) تمرین‌ها

۱) **دیابت:** یک ویژگی نشت‌دار دوم بسازید که شبیه «نشانگر پس از تشخیص» باشد (مثلاً فلگی مشتق از هدف) و ببینید تشخیص داده می‌شود یا نه.  
۲) **قیمت خانه:** یک ویژگی «باکت نشت‌دار» بسازید: `PriceBucket = round(Price / 10000)` و تورم عملکرد را اندازه بگیرید.  
۳) **لیستینگ‌ها:** آزمون حساسیت به split را بعد از حذف `host_id` و ستون‌های مرتبط با review دوباره اجرا کنید. CV گروهی چقدر تغییر می‌کند؟  
۴) **شکایات:** $t_0$ را به «بعد از ارسال شکایت به شرکت» تغییر دهید و تصمیم بگیرید کدام ستون‌های گردش‌کار معتبر می‌شوند. نتایج چگونه تغییر می‌کند؟

وقتی بتوانید به این پرسش‌ها پاسخ دهید، از «دانستن اینکه نشت وجود دارد» به توانایی «اثبات پزشکی-قانونی نشت و پیشگیری از آن» رسیده‌اید.

## ۱۱) جمع‌بندی

- نشت نقض مرز در دسترس بودن است: ویژگی‌ها (یا پیش‌پردازش/ارزیابی) شامل اطلاعات ممنوع می‌شوند.
- EDA بهترین زمان برای کشف نشت است چون در این مرحله معنی ستون‌ها و مسائل lineage روشن می‌شوند.
- بررسی معنایی را با غربال‌های کمی ترکیب کنید:
  - نام‌های مشکوک،
  - عملکرد تک‌ویژگی غیرعادی،
  - حساسیت به split (تصادفی/گروهی/زمانی)،
  - آزمایش‌های کنترل‌شدهٔ «اضافه/حذف ستون».
- قرارداد as-of time را صریح کنید. ستون‌های گردش‌کارِ پسینی رایج‌ترین منبع نشت در داده‌های واقعی هستند.
- بهداشت پیش‌پردازش را با پایپلاین و فیت فقط روی آموزش enforce کنید.

در درس بعدی یاد می‌گیرید چگونه خروجی EDA را به گزارشی تبدیل کنید که برای ذی‌نفعان فنی و غیرفنی قابل اقدام باشد.